In [1]:
import geopandas as gpd
from shapely.geometry import Point, Polygon
from scipy import spatial
import numpy as np
import pandas as pd

In [2]:
#load índice bienestar territorial 2012
ibt = gpd.read_file('data/ibt_geo.shp', crs={'init': 'epsg:5361'})
ibt = ibt.dropna(subset = ['nse']).reset_index(drop = True)
ibt.tail()

,comuna,ibt,iseg,nse,geometry
40721,PUENTE ALTO,30.0,28.45,75.71,"POLYGON ((-70.54646 -33.59121, -70.54651 -33.5..."
40722,PUENTE ALTO,20.0,28.45,75.71,"POLYGON ((-70.54512 -33.59019, -70.54514 -33.5..."
40723,PUENTE ALTO,20.0,28.45,75.71,"POLYGON ((-70.54406 -33.59091, -70.54436 -33.5..."
40724,PUENTE ALTO,30.0,28.45,75.71,"POLYGON ((-70.54362 -33.59163, -70.54372 -33.5..."
40725,PUENTE ALTO,20.0,28.45,75.71,"POLYGON ((-70.54555 -33.59051, -70.54557 -33.5..."


In [3]:
# calculate the centroid of each block
mcentroids = ibt['geometry'].centroid

In [4]:
# Create KDTree to consult the closest block per point
data = list(zip(mcentroids.x.ravel(), mcentroids.y.ravel()))
tree = spatial.KDTree(data = data, leafsize = 10)

In [5]:
# Import lat lon data
georef = pd.read_csv('locs_geo_toro.csv', encoding = 'utf-8')
georef.head()

,Unnamed: 0,post_id,loc,loc_unique_match,lat,lon,type
0,1,FB_5574,"Principe de Gales esq Tobalaba, La Reina, Regi...",1,-33.437832,-70.573589,GEOMETRIC_CENTER
1,2,FB_5577,"Presidente Riesco esq Vitacura, Las Condes, Re...",2,-33.411530,-70.602785,GEOMETRIC_CENTER
2,3,FB_5592,"Gimnasio municipal de Lo Barnechea, Lo Barnech...",3,-33.358367,-70.507986,ROOFTOP
3,4,FB_5595,"Unión Comunal del Adulto Mayor, Las Condes , R...",4,-33.409212,-70.556373,ROOFTOP
4,5,FB_5614,"Centro de Adulto Mayor, La Reina, Region Metro...",5,-33.439884,-70.566425,ROOFTOP


In [9]:
# Isolate lat lon points for KDTree queries
pts = list(zip(georef['lon'].ravel(), georef['lat'].ravel()))

In [7]:
%%time
# Calculate the 2 nearest blocks per lat lon point
kind = tree.query(pts, k = 2)[1]

CPU times: user 29.2 ms, sys: 703 µs, total: 29.9 ms
Wall time: 22.7 ms


In [8]:
%%time
# Calculate average NSE per block pair
georef.loc[:,('nse')] = list(map(lambda i: ibt.loc[kind[i],'nse'].mean(), range(len(pts))))

CPU times: user 49.4 ms, sys: 15.2 ms, total: 64.6 ms
Wall time: 47 ms


In [23]:
#output table, with lat lon data and an associated NSE
georef.to_csv('posts_loc_toro.csv', encoding = 'utf-8', index = False)